In [25]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')
from sklearn import svm, cross_validation
import pylab as pl
from PIL import Image
import numpy as np
import os

In [26]:
# Make sure that caffe is on the python path:
caffe_root = '/u/zexuan/caffe/caffe/'  # this file is expected to be in {caffe_root}/examples
caffe_real_root = '/pkgs/caffe/'
thesis_root = '/ais/gobi2/pingpong/thesis/'
#!ls /pkgs/caffe
import sys
sys.path.insert(0, caffe_real_root + 'python')
import caffe

plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

if not os.path.isfile('/u/zexuan/caffe/caffe/models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel'):
    print("Downloading pre-trained CaffeNet model...")
    !~/caffe/caffe/scripts/download_model_binary.py ~/caffe/caffe/models/bvlc_reference_caffenet

In [27]:
caffe.set_device(0)
caffe.set_mode_gpu()

In [28]:
# MODEL_FILE = caffe_root +'models/bvlc_reference_caffenet/deploy.prototxt'
# PRETRAINED = caffe_root +'models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel'
# caffe.set_mode_cpu()
# net = caffe.Classifier(MODEL_FILE, PRETRAINED,
#                        mean=np.load(caffe_root + 'python/caffe/imagenet/ilsvrc_2012_mean.npy').mean(1).mean(1),
#                        channel_swap=(2,1,0),
#                        raw_scale=255,
#                        image_dims=(600, 600))
# print net.blobs['data'].data.shape
# [(k, v.data.shape) for k, v in net.blobs.items()]


# caffe.set_mode_cpu()
print caffe_root + 'models/bvlc_reference_caffenet/deploy.prototxt'
#!ls caffe_root+'models/bvlc_reference_caffenet/deploy.prototxt'
# net = caffe.Net(caffe_root + 'models/bvlc_reference_caffenet/deploy.prototxt',
#                 caffe_root + 'models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel',
#                 caffe.TEST)

#~/caffe/caffe/models/finetune_flickr_style/deploy.prototxt
net = caffe.Net(caffe_root + 'models/finetune_stroke_cls/deploy.prototxt',
                thesis_root + 'fine_tune_stroke_cls_2000iter.caffemodel',
                caffe.TEST)

# input preprocessing: 'data' is the name of the input blob == net.inputs[0]
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
transformer.set_transpose('data', (2,0,1))
transformer.set_mean('data', np.load(caffe_root + 'python/caffe/imagenet/ilsvrc_2012_mean.npy').mean(1).mean(1)) # mean pixel
transformer.set_raw_scale('data', 255)  # the reference model operates on images in [0,255] range instead of [0,1]
transformer.set_channel_swap('data', (2,1,0))  # the reference model has channels in BGR order instead of RGB
print net.blobs['data'].data.shape
[(k, v.data.shape) for k, v in net.blobs.items()] 

/u/zexuan/caffe/caffe/models/bvlc_reference_caffenet/deploy.prototxt
(10, 3, 227, 227)


[('data', (10, 3, 227, 227)),
 ('conv1', (10, 96, 55, 55)),
 ('pool1', (10, 96, 27, 27)),
 ('norm1', (10, 96, 27, 27)),
 ('conv2', (10, 256, 27, 27)),
 ('pool2', (10, 256, 13, 13)),
 ('norm2', (10, 256, 13, 13)),
 ('conv3', (10, 384, 13, 13)),
 ('conv4', (10, 384, 13, 13)),
 ('conv5', (10, 256, 13, 13)),
 ('pool5', (10, 256, 6, 6)),
 ('fc6', (10, 4096)),
 ('fc7', (10, 4096)),
 ('fc8_flickr', (10, 20)),
 ('prob', (10, 20))]

In [29]:
import skimage
import skimage.io
import time

start_time = time.time()
training_set = {}
training_set['target'] = []
container_path = 'labeled_img_selected_all_classes/'

folders = [f for f in sorted(os.listdir(container_path)) if os.path.isdir(os.path.join(container_path, f))]
print 'folders: ', folders

result = []
for folder in folders:
    folder_path = os.path.join(container_path, folder)
    documents = [os.path.join(folder_path, d) for d in sorted(os.listdir(folder_path))]
    input_10_imgs = []
    for pic in documents:
        if not pic.endswith('png'):
            continue
        img = skimage.img_as_float(skimage.io.imread(pic)).astype(np.float32)
        input_10_imgs.append(img)
        if len(input_10_imgs) >= 10:
            #net.blobs['data'].data[...] = transformer.preprocess('data', training_set['data'][0])
            #net.blobs['data'].data[...] = map(lambda x: transformer.preprocess('data', x), training_set['data'][i:i+10])
            net.blobs['data'].data[...] = map(lambda x: transformer.preprocess('data', x), input_10_imgs)
            out = net.forward()
            result.append(np.mean(net.blobs['fc6'].data, axis=0))
            input_10_imgs = []
        training_set['target'].append(folder)
#training_set['data']=data
#training_set['target']=target
print 'Total time loading training data: ', time.time()-start_time, ' seconds'
print 'number of images: ', len(training_set['target'])
#print 'dmensionality of each image: ', input_10_imgs[0].shape

folders:  ['C10_TOP_PLAYER_BACKHAND_BLOCK_selected', 'C11_BOTTOM_PLAYER_FOREHAND_BLOCK_selected', 'C12_BOTTOM_PLAYER_BACKHAND_BLOCK_selected', 'C13_TOP_PLAYER_FOREHAND_FLIP_selected', 'C14_TOP_PLAYER_BACKHAND_FLIP_selected', 'C15_BOTTOM_PLAYER_FOREHAND_FLIP_selected', 'C16_BOTTOM_PLAYER_BACKHAND_FLIP_selected', 'C17_TOP_PLAYER_FOREHAND_CHOP_selected', 'C18_TOP_PLAYER_BACKHAND_CHOP_selected', 'C19_BOTTOM_PLAYER_FOREHAND_CHOP_selected', 'C1_TOP_PLAYER_FOREHAND_SERVE_selected', 'C20_BOTTOM_PLAYER_BACKHAND_CHOP_selected', 'C2_TOP_PLAYER_BACKHAND_SERVE_selected', 'C3_BOTTOM_PLAYER_FOREHAND_SERVE_selected', 'C4_BOTTOM_PLAYER_BACKHAND_SERVE_selected', 'C5_TOP_PLAYER_FOREHAND_LOOP_selected', 'C6_TOP_PLAYER_BACKHAND_LOOP_selected', 'C7_BOTTOM_PLAYER_FOREHAND_LOOP_selected', 'C8_BOTTOM_PLAYER_BACKHAND_LOOP_selected', 'C9_TOP_PLAYER_FOREHAND_BLOCK_selected']
Total time loading training data:  475.210674047  seconds
number of images:  11680


In [30]:
start_time = time.time()
testing_set = {}
testing_set['target'] = []
container_path = 'labeled_img_selected_all_classes_testset/'

folders = [f for f in sorted(os.listdir(container_path)) if os.path.isdir(os.path.join(container_path, f))]
print 'folders: ', folders

for folder in folders:
    folder_path = os.path.join(container_path, folder)
    documents = [os.path.join(folder_path, d) for d in sorted(os.listdir(folder_path))]
    input_10_imgs = []
    for pic in documents:
        if not pic.endswith('png'):
            continue
        img = skimage.img_as_float(skimage.io.imread(pic)).astype(np.float32)
        input_10_imgs.append(img)
        if len(input_10_imgs) >= 10:
            #net.blobs['data'].data[...] = transformer.preprocess('data', training_set['data'][0])
            #net.blobs['data'].data[...] = map(lambda x: transformer.preprocess('data', x), training_set['data'][i:i+10])
            net.blobs['data'].data[...] = map(lambda x: transformer.preprocess('data', x), input_10_imgs)
            out = net.forward()
            result.append(np.mean(net.blobs['fc6'].data, axis=0))
            input_10_imgs = []
        testing_set['target'].append(folder)
#training_set['data']=data
#training_set['target']=target
print 'Total time loading training data: ', time.time()-start_time, ' seconds'
print 'number of images: ', len(testing_set['target'])

folders:  ['C10_TOP_PLAYER_BACKHAND_BLOCK_selected', 'C11_BOTTOM_PLAYER_FOREHAND_BLOCK_selected', 'C12_BOTTOM_PLAYER_BACKHAND_BLOCK_selected', 'C13_TOP_PLAYER_FOREHAND_FLIP_selected', 'C14_TOP_PLAYER_BACKHAND_FLIP_selected', 'C15_BOTTOM_PLAYER_FOREHAND_FLIP_selected', 'C16_BOTTOM_PLAYER_BACKHAND_FLIP_selected', 'C17_TOP_PLAYER_FOREHAND_CHOP_selected', 'C18_TOP_PLAYER_BACKHAND_CHOP_selected', 'C19_BOTTOM_PLAYER_FOREHAND_CHOP_selected', 'C1_TOP_PLAYER_FOREHAND_SERVE_selected', 'C20_BOTTOM_PLAYER_BACKHAND_CHOP_selected', 'C2_TOP_PLAYER_BACKHAND_SERVE_selected', 'C3_BOTTOM_PLAYER_FOREHAND_SERVE_selected', 'C4_BOTTOM_PLAYER_BACKHAND_SERVE_selected', 'C5_TOP_PLAYER_FOREHAND_LOOP_selected', 'C6_TOP_PLAYER_BACKHAND_LOOP_selected', 'C7_BOTTOM_PLAYER_FOREHAND_LOOP_selected', 'C8_BOTTOM_PLAYER_BACKHAND_LOOP_selected', 'C9_TOP_PLAYER_FOREHAND_BLOCK_selected']
Total time loading training data:  101.528191805  seconds
number of images:  2500


In [31]:
# print caffe.io.load_image(thesis_root + 'labeled_image_selected/bottom_player_winning_selected/point_00001_frame_00033.png').shape
# print caffe.io.load_image(thesis_root + 'labeled_image_selected/bottom_player_winning_selected/point_00001_frame_00033.png')[0][0]
# print training_set['data'][0][0][0]

'''
result = []
#for i in xrange(2200, 2300,10):
for i in xrange(0, traning_set_size, 10):
    #net.blobs['data'].data[...] = transformer.preprocess('data', training_set['data'][0])
    net.blobs['data'].data[...] = map(lambda x: transformer.preprocess('data', x), training_set['data'][i:i+10])
    out = net.forward()
    #print net.blobs['fc7'].data.shape
    result.append(np.mean(net.blobs['fc6'].data, axis=0))
'''

print 'Number of input to SVM: ', len(result), 'Size of each input', result[0].shape

print transformer.deprocess('data', net.blobs['data'].data[0]).shape
print 'Sample image from the training set'
#plt.imshow(transformer.deprocess('data', net.blobs['data'].data[4]))

Number of input to SVM:  1418 Size of each input (4096,)
(227, 227, 3)
Sample image from the training set


In [32]:
import pickle

pickle.dump(result, open("nn_result_fc6_unnormalized_clf_7videos.pickle", "wb" ) )

In [33]:
#from sklearn.preprocessing import normalize
print len(result)
print np.max(result[0])
print result[0].shape
normalized_result = []
#normalized_result = normalize(result)
for r in result:
    #normalized_result.append(r/np.max(r))
    normalized_result.append((r-np.mean(r))/np.std(r))

1418
9.84746
(4096,)


In [34]:
import pickle

pickle.dump(normalized_result, open("nn_result_fc6_normalized_clf_7videos.pickle", "wb" ) )

In [35]:
print len(normalized_result)
print np.max(normalized_result[0])
print normalized_result[0].shape
print result[0].shape


1418
7.75668
(4096,)
(4096,)


In [36]:
from sklearn import decomposition

traning_set_size = len(training_set['target'])
testing_set_size = len(testing_set['target'])
print 'traning_set_size: ', traning_set_size
print 'normalized_result size: ', len(normalized_result)
#X_train, X_test, y_train, y_test = cross_validation.train_test_split(result, training_set['target'][0:traning_set_size:10], test_size=0.25, random_state=42)
#X_train, X_test, y_train, y_test = cross_validation.train_test_split(normalized_result, training_set['target'][0:traning_set_size:10], test_size=0.15, random_state=2)
X_train = normalized_result[:traning_set_size/10]
X_test = normalized_result[traning_set_size/10:]
#X_train = result[:traning_set_size/10]
#X_test = result[traning_set_size/10:]
y_train = training_set['target'][0:traning_set_size:10]
y_test = testing_set['target'][0:traning_set_size:10]


print 'X_train: %d; X_test: %d; y_train: %d; y_test: %d'%(len(X_train), len(X_test), len(y_train), len(y_test))

traning_set_size:  11680
normalized_result size:  1418
X_train: 1168; X_test: 250; y_train: 1168; y_test: 250


In [37]:
from sklearn import svm, cross_validation
X_train = np.asarray(X_train)
svc = svm.SVC(kernel='precomputed')
kernel_train = np.dot(X_train, X_train.T)  # linear kernel
svc.fit(kernel_train, y_train)

# Testing
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
kernel_test = np.dot(X_test, X_train.T)
y_pred = svc.predict(kernel_test)
print accuracy_score(y_test, y_pred)

import pandas as pd
y_true = pd.Series(y_test)
y_pred = pd.Series(y_pred)

#print pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

print confusion_matrix(y_test, y_pred).shape
print confusion_matrix(y_test, y_pred)
#print '\n'.join(sorted(list(set(y_test))))
print len(set(y_test).union(set(y_pred)))

0.692
(18, 18)
[[ 3  0  0  0  1  0  0  0  0  2  0  0  0  1  1  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0]
 [ 0  0  1  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0]
 [ 0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0]
 [ 1  0  0  0  6  0  0  1  1  0  0  0  0  1  1  1  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  1  0  0  1  1  0  0]
 [ 0  0  0  0  6  0  0 10  0  2  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  2  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  5  0  0  0  8  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  3 41  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  2  0  0  0  1  0  0]
 [ 0  0  0  0  0  0  0  0  0  2  0  0 62  0  0  0  0  0]
 [ 0  0  1  0  0  0  0  2  0  4  0  0  0 19  0  1  1  0]
 [ 4  0  0  0  0  0  0  1  2  1  0  0  0  8  3  0  1  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  4  0 14  0  0]
 [ 0  0  0  0  0  0  0  0  0  3  0  0  0  3  1  2  2  0]
 [ 0  0  0  0  0

In [38]:
for c in [0.001, 0.005, 0.01, 0.05, 0.1, 1, 10, 500]:
    svc = svm.SVC(kernel='linear', C=c) 
    kfold = cross_validation.KFold(len(X_train), n_folds=4, shuffle=True)
    cv_result = cross_validation.cross_val_score(svc, X_train, y_train, cv=kfold, n_jobs=1)
    print 'C = %g, Cross validation: %s'%(c, cv_result)
    print 'C = %g, Mean value: %g'%(c, np.mean(cv_result))

C = 0.001, Cross validation: [ 0.89383562  0.86986301  0.8869863   0.91438356]
C = 0.001, Mean value: 0.891267
C = 0.005, Cross validation: [ 0.87328767  0.90753425  0.8869863   0.85273973]
C = 0.005, Mean value: 0.880137
C = 0.01, Cross validation: [ 0.85958904  0.85958904  0.90068493  0.87328767]
C = 0.01, Mean value: 0.873288
C = 0.05, Cross validation: [ 0.89383562  0.86986301  0.85273973  0.82876712]
C = 0.05, Mean value: 0.861301
C = 0.1, Cross validation: [ 0.84931507  0.85616438  0.84589041  0.85273973]
C = 0.1, Mean value: 0.851027
C = 1, Cross validation: [ 0.82876712  0.85616438  0.86986301  0.87671233]
C = 1, Mean value: 0.857877
C = 10, Cross validation: [ 0.88013699  0.8630137   0.85616438  0.84589041]
C = 10, Mean value: 0.861301
C = 500, Cross validation: [ 0.81164384  0.87671233  0.84589041  0.85958904]
C = 500, Mean value: 0.848459


In [39]:
for c in [0.01, 0.05, 0.1, 1, 10, 100, 1000, 1000000]:
    svc = svm.SVC(C=c, kernel='poly', degree=1)
    kfold = cross_validation.KFold(len(X_train), n_folds=4, shuffle=True)
    cv_result = cross_validation.cross_val_score(svc, X_train, y_train, cv=kfold, n_jobs=1)
    print 'C = %g, Cross validation: %s'%(c, cv_result)
    print 'C = %g, Mean value: %g'%(c, np.mean(cv_result))

C = 0.01, Cross validation: [ 0.17808219  0.31849315  0.18493151  0.25684932]
C = 0.01, Mean value: 0.234589
C = 0.05, Cross validation: [ 0.59589041  0.59589041  0.57534247  0.63013699]
C = 0.05, Mean value: 0.599315
C = 0.1, Cross validation: [ 0.73972603  0.72260274  0.73972603  0.68835616]
C = 0.1, Mean value: 0.722603
C = 1, Cross validation: [ 0.85273973  0.8869863   0.86643836  0.90753425]
C = 1, Mean value: 0.878425
C = 10, Cross validation: [ 0.88013699  0.87671233  0.85273973  0.87671233]
C = 10, Mean value: 0.871575
C = 100, Cross validation: [ 0.86643836  0.87671233  0.88356164  0.83219178]
C = 100, Mean value: 0.864726
C = 1000, Cross validation: [ 0.85958904  0.84246575  0.86986301  0.82876712]
C = 1000, Mean value: 0.850171
C = 1e+06, Cross validation: [ 0.84246575  0.86643836  0.87328767  0.83219178]
C = 1e+06, Mean value: 0.853596


In [40]:
for c in [0.01, 0.05, 0.1, 1, 10, 100, 1000, 1000000]:
    svc = svm.SVC(C=c, kernel='poly', degree=2)
    kfold = cross_validation.KFold(len(X_train), n_folds=4, shuffle=True)
    cv_result = cross_validation.cross_val_score(svc, X_train, y_train, cv=kfold, n_jobs=1)
    print 'C = %g, Cross validation: %s'%(c, cv_result)
    print 'C = %g, Mean value: %g'%(c, np.mean(cv_result))

C = 0.01, Cross validation: [ 0.20205479  0.19178082  0.18835616  0.2260274 ]
C = 0.01, Mean value: 0.202055
C = 0.05, Cross validation: [ 0.50342466  0.51027397  0.48287671  0.54452055]
C = 0.05, Mean value: 0.510274
C = 0.1, Cross validation: [ 0.64726027  0.59246575  0.5890411   0.65410959]
C = 0.1, Mean value: 0.620719
C = 1, Cross validation: [ 0.87328767  0.85616438  0.86643836  0.85616438]
C = 1, Mean value: 0.863014
C = 10, Cross validation: [ 0.85616438  0.81506849  0.8390411   0.8869863 ]
C = 10, Mean value: 0.849315
C = 100, Cross validation: [ 0.8630137   0.85616438  0.84931507  0.8390411 ]
C = 100, Mean value: 0.851884
C = 1000, Cross validation: [ 0.85273973  0.80479452  0.84246575  0.85273973]
C = 1000, Mean value: 0.838185
C = 1e+06, Cross validation: [ 0.85273973  0.84246575  0.82876712  0.84246575]
C = 1e+06, Mean value: 0.84161


In [41]:
for c in [0.01, 0.05, 0.1, 1, 10, 100, 1000, 1000000]:
    svc = svm.SVC(C=c, kernel='poly', degree=3)
    kfold = cross_validation.KFold(len(X_train), n_folds=4, shuffle=True)
    cv_result = cross_validation.cross_val_score(svc, X_train, y_train, cv=kfold, n_jobs=1)
    print 'C = %g, Cross validation: %s'%(c, cv_result)
    print 'C = %g, Mean value: %g'%(c, np.mean(cv_result))

C = 0.01, Cross validation: [ 0.21575342  0.21575342  0.20547945  0.17123288]
C = 0.01, Mean value: 0.202055
C = 0.05, Cross validation: [ 0.4760274   0.46917808  0.39726027  0.40068493]
C = 0.05, Mean value: 0.435788
C = 0.1, Cross validation: [ 0.46232877  0.53424658  0.54109589  0.52054795]
C = 0.1, Mean value: 0.514555
C = 1, Cross validation: [ 0.84589041  0.82876712  0.8630137   0.80136986]
C = 1, Mean value: 0.83476
C = 10, Cross validation: [ 0.84246575  0.82876712  0.83219178  0.79794521]
C = 10, Mean value: 0.825342
C = 100, Cross validation: [ 0.80821918  0.85273973  0.79452055  0.79452055]
C = 100, Mean value: 0.8125
C = 1000, Cross validation: [ 0.82191781  0.8390411   0.83561644  0.80821918]
C = 1000, Mean value: 0.826199
C = 1e+06, Cross validation: [ 0.82876712  0.82534247  0.82876712  0.80479452]
C = 1e+06, Mean value: 0.821918


In [42]:
clf_best = svm.SVC(kernel='linear', C=0.01).fit(X_train, y_train)
clf_best.score(X_test, y_test)

0.72799999999999998

In [43]:
print len(X_test)
print len(y_test)
for i,_ in enumerate(X_test):
    if clf_best.predict(X_test[i])[0] == y_test[i]:
        print 'SAME: ', clf_best.predict(X_test[i])[0]
    else:
        print 'DIFFERENT: ', clf_best.predict(X_test[i])[0], y_test[i]
        

250
250
DIFFERENT:  C14_TOP_PLAYER_BACKHAND_FLIP_selected C10_TOP_PLAYER_BACKHAND_BLOCK_selected
SAME:  C10_TOP_PLAYER_BACKHAND_BLOCK_selected
DIFFERENT:  C19_BOTTOM_PLAYER_FOREHAND_CHOP_selected C10_TOP_PLAYER_BACKHAND_BLOCK_selected
DIFFERENT:  C6_TOP_PLAYER_BACKHAND_LOOP_selected C10_TOP_PLAYER_BACKHAND_BLOCK_selected
DIFFERENT:  C19_BOTTOM_PLAYER_FOREHAND_CHOP_selected C10_TOP_PLAYER_BACKHAND_BLOCK_selected
SAME:  

/u/zexuan/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/u/zexuan/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/u/zexuan/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationW

C10_TOP_PLAYER_BACKHAND_BLOCK_selected
SAME:  C10_TOP_PLAYER_BACKHAND_BLOCK_selected
DIFFERENT:  C5_TOP_PLAYER_FOREHAND_LOOP_selected C10_TOP_PLAYER_BACKHAND_BLOCK_selected
DIFFERENT:  C5_TOP_PLAYER_FOREHAND_LOOP_selected C11_BOTTOM_PLAYER_FOREHAND_BLOCK_selected
SAME:  C12_BOTTOM_PLAYER_BACKHAND_BLOCK_selected
DIFFERENT:  C7_BOTTOM_PLAYER_FOREHAND_LOOP_selected C12_BOTTOM_PLAYER_BACKHAND_BLOCK_selected
DIFFERENT:  

/u/zexuan/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/u/zexuan/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/u/zexuan/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationW

C17_TOP_PLAYER_FOREHAND_CHOP_selected C13_TOP_PLAYER_FOREHAND_FLIP_selected
SAME:  C14_TOP_PLAYER_BACKHAND_FLIP_selected
DIFFERENT:  C19_BOTTOM_PLAYER_FOREHAND_CHOP_selected C14_TOP_PLAYER_BACKHAND_FLIP_selected
SAME:  C14_TOP_PLAYER_BACKHAND_FLIP_selected
SAME:  C14_TOP_PLAYER_BACKHAND_FLIP_selected
DIFFERENT:  C7_BOTTOM_PLAYER_FOREHAND_LOOP_selected C14_TOP_PLAYER_BACKHAND_FLIP_selected
SAME:  

...

/u/zexuan/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/u/zexuan/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/u/zexuan/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationW

 C7_BOTTOM_PLAYER_FOREHAND_LOOP_selected
SAME:  C7_BOTTOM_PLAYER_FOREHAND_LOOP_selected
SAME:  C7_BOTTOM_PLAYER_FOREHAND_LOOP_selected
SAME:  C8_BOTTOM_PLAYER_BACKHAND_LOOP_selected
SAME:  C8_BOTTOM_PLAYER_BACKHAND_LOOP_selected
DIFFERENT:  C19_BOTTOM_PLAYER_FOREHAND_CHOP_selected C8_BOTTOM_PLAYER_BACKHAND_LOOP_selected
DIFFERENT:  C19_BOTTOM_PLAYER_FOREHAND_CHOP_selected C8_BOTTOM_PLAYER_BACKHAND_LOOP_selected
DIFFERENT: 

/u/zexuan/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/u/zexuan/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/u/zexuan/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationW

 C19_BOTTOM_PLAYER_FOREHAND_CHOP_selected C8_BOTTOM_PLAYER_BACKHAND_LOOP_selected
DIFFERENT:  C19_BOTTOM_PLAYER_FOREHAND_CHOP_selected C8_BOTTOM_PLAYER_BACKHAND_LOOP_selected
DIFFERENT:  C7_BOTTOM_PLAYER_FOREHAND_LOOP_selected C8_BOTTOM_PLAYER_BACKHAND_LOOP_selected
DIFFERENT:  C7_BOTTOM_PLAYER_FOREHAND_LOOP_selected C8_BOTTOM_PLAYER_BACKHAND_LOOP_selected
DIFFERENT:  C7_BOTTOM_PLAYER_FOREHAND_LOOP_selected C8_BOTTOM_PLAYER_BACKHAND_LOOP_selected
DIFFERENT:  C5_TOP_PLAYER_FOREHAND_LOOP_selected C8_BOTTOM_PLAYER_BACKHAND_LOOP_selected
DIFFERENT:  C5_TOP_PLAYER_FOREHAND_LOOP_selected C8_BOTTOM_PLAYER_BACKHAND_LOOP_selected
DIFFERENT:  C5_TOP_PLAYER_FOREHAND_LOOP_selected C9_TOP_PLAYER_FOREHAND_BLOCK_selected
DIFFERENT:  C5_TOP_PLAYER_FOREHAND_LOOP_selected C9_TOP_PLAYER_FOREHAND_BLOCK_selected


/u/zexuan/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/u/zexuan/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/u/zexuan/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationW

In [44]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=7)

kfold = cross_validation.KFold(len(X_train), n_folds=4, shuffle=True)
cv_result = cross_validation.cross_val_score(neigh, X_train, y_train, cv=kfold, n_jobs=1)

print 'Cross validataion result', cv_result, '. Avg:', np.mean(cv_result)
    
clf = neigh.fit(X_train, y_train)
print 'Testing result', clf.score(X_test, y_test)

Cross validataion result [ 0.76712329  0.77739726  0.7739726   0.72945205] . Avg: 0.76198630137
Testing result 0.64


In [45]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
cv_result = cross_validation.cross_val_score(gnb, X_train, y_train, cv=kfold, n_jobs=1)
print 'Cross validataion result', cv_result, '. Avg:', np.mean(cv_result)

clf = gnb.fit(X_train, y_train)
print 'Testing result', clf.score(X_test, y_test)

Cross validataion result [ 0.82876712  0.84246575  0.82876712  0.80821918] . Avg: 0.827054794521
Testing result 0.464


In [46]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
cv_result = cross_validation.cross_val_score(lr, X_train, y_train, cv=kfold, n_jobs=1)
print 'Cross validataion result', cv_result, '. Avg:', np.mean(cv_result)

clf = lr.fit(X_train, y_train)
print 'Testing result', clf.score(X_test, y_test)

Cross validataion result [ 0.86643836  0.92123288  0.90753425  0.85273973] . Avg: 0.88698630137
Testing result 0.704


In [47]:
pickle.dump(clf, open("fc6_normalized_lr_clf_7videos_2.pickle", "wb" ) )

In [48]:
print sorted(list(set(y_train)))
print len(sorted(list(set(y_train))))

['C10_TOP_PLAYER_BACKHAND_BLOCK_selected', 'C11_BOTTOM_PLAYER_FOREHAND_BLOCK_selected', 'C12_BOTTOM_PLAYER_BACKHAND_BLOCK_selected', 'C13_TOP_PLAYER_FOREHAND_FLIP_selected', 'C14_TOP_PLAYER_BACKHAND_FLIP_selected', 'C15_BOTTOM_PLAYER_FOREHAND_FLIP_selected', 'C16_BOTTOM_PLAYER_BACKHAND_FLIP_selected', 'C17_TOP_PLAYER_FOREHAND_CHOP_selected', 'C18_TOP_PLAYER_BACKHAND_CHOP_selected', 'C19_BOTTOM_PLAYER_FOREHAND_CHOP_selected', 'C1_TOP_PLAYER_FOREHAND_SERVE_selected', 'C20_BOTTOM_PLAYER_BACKHAND_CHOP_selected', 'C3_BOTTOM_PLAYER_FOREHAND_SERVE_selected', 'C5_TOP_PLAYER_FOREHAND_LOOP_selected', 'C6_TOP_PLAYER_BACKHAND_LOOP_selected', 'C7_BOTTOM_PLAYER_FOREHAND_LOOP_selected', 'C8_BOTTOM_PLAYER_BACKHAND_LOOP_selected', 'C9_TOP_PLAYER_FOREHAND_BLOCK_selected']
18
